In [29]:
import pandas as pd
import numpy as np
from influxdb_client import InfluxDBClient
from sklearn.impute import IterativeImputer

# InfluxDB 연결
def get_influx_client():
    return InfluxDBClient(
        url="https://influx.aiot2.live",
        token="RmaabELI9VpYPRu4nt_xBZX5l3Gv5lx8XnR4mVZnqep4Ya3eYrfpLUk4Y4dYE4J0mlcFHFPLUCKh8a4jq_lMNw==",
        org="aiot2-team2-coffee"
    ), "coffee-mqtt"

# 센서 및 장소 설정
sensor_types = ["temperature", "humidity", "co2"]
locations = ["8인 책상", "입구", "안쪽벽 중앙", "보드"]

# Flux 쿼리
client, bucket = get_influx_client()
query = client.query_api()
sensor_filter = ' or '.join([f'r["type"] == "{t}"' for t in sensor_types])
location_filter = ' or '.join([f'r["location"] == "{loc}"' for loc in locations])
flux = f'''
from(bucket: "{bucket}")
  |> range(start: -8d)
  |> filter(fn: (r) => r["_measurement"] == "sensor")
  |> filter(fn: (r) => r["_field"] == "value")
  |> filter(fn: (r) => {sensor_filter})
  |> filter(fn: (r) => {location_filter})
  |> aggregateWindow(every: 5m, fn: mean)
  |> yield(name: "mean")
'''

# 데이터 수집 및 전처리
df = query.query_data_frame(flux)
if isinstance(df, list): df = pd.concat(df, ignore_index=True)
df['time'] = pd.to_datetime(df['_time'])
pivot = df.pivot_table(index=['time', 'location'], columns='type', values='_value').reset_index()

# 결측치 보정
imp = IterativeImputer()
pivot[['temperature', 'humidity', 'co2']] = imp.fit_transform(pivot[['temperature', 'humidity', 'co2']])

# 면적 및 인원수 추정
area_map = {"8인 책상": 64.65, "입구": 109.21, "안쪽벽 중앙": 25.13, "보드": 31.59}
pivot['area'] = pivot['location'].map(area_map)
pivot['people_est'] = ((pivot['co2'] - 400) / 250) * (pivot['area'] / 30)
pivot['people_est'] = pivot['people_est'].clip(lower=0).round(1)

# 결과
print(pivot[['time', 'location', 'temperature', 'humidity', 'co2', 'area', 'people_est']].head())


/opt/homebrew/anaconda3/lib/python3.12/site-packages/influxdb_client/client/warnings.py:31: MissingPivotFunction: The query doesn't contains the pivot() function.

The result will not be shaped to optimal processing by pandas.DataFrame. Use the pivot() function by:

    
from(bucket: "coffee-mqtt")
  |> range(start: -8d)
  |> filter(fn: (r) => r["_measurement"] == "sensor")
  |> filter(fn: (r) => r["_field"] == "value")
  |> filter(fn: (r) => r["type"] == "temperature" or r["type"] == "humidity" or r["type"] == "co2")
  |> filter(fn: (r) => r["location"] == "8인 책상" or r["location"] == "입구" or r["location"] == "안쪽벽 중앙" or r["location"] == "보드")
  |> aggregateWindow(every: 5m, fn: mean)
  |> yield(name: "mean")
 |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")

You can disable this warning by:
    import warnings
    from influxdb_client.client.warnings import MissingPivotFunction

    warnings.simplefilter("ignore", MissingPivotFunction)

For more info see:
    -

type                      time location  temperature  humidity     co2  \
0    2025-04-24 08:05:00+00:00    8인 책상        23.40     51.50  512.00   
1    2025-04-24 08:05:00+00:00       보드        25.00     49.00  701.00   
2    2025-04-24 08:05:00+00:00       입구        12.20     87.25  840.00   
3    2025-04-24 08:10:00+00:00    8인 책상        23.30     51.50  507.75   
4    2025-04-24 08:10:00+00:00       보드        24.96     49.20  699.00   

type    area  people_est  
0      64.65         1.0  
1      31.59         1.3  
2     109.21         6.4  
3      64.65         0.9  
4      31.59         1.3  


In [31]:
pivot.describe()

type,co2,humidity,temperature,area,people_est
count,7988.000000,7988.000000,7988.000000,7988.000000,7988.000000
mean,623.445560,37.029687,23.819210,62.451510,2.134251
std,167.882349,8.225349,1.452537,33.117882,2.606009
min,412.000000,17.800000,10.755556,25.130000,0.100000
25%,500.000000,31.500000,23.400000,31.590000,0.600000
50%,588.000000,36.000000,24.000000,64.650000,1.100000
75%,685.233333,40.500000,24.392308,109.210000,2.500000
max,1350.000000,92.000000,26.600000,109.210000,13.800000


In [33]:
pivot.to_csv("./people_estimated_dataset.csv", index=False)